# KSA: Polar Observations and Modelling

# Week 6 - Introduction to an ocean/sea ice model

Now that we have done a little overview of ACCESS-OM2 and some of its data, it's time to do a bit of exploring into this model output. 

For this, you can find in the `data/` directory, output for the Southern Ocean that includes temperature and salinity fields, sea surface height, and surface currents. All this variables come in monthly averages, from 1980 to 2018. We have also included a bit of information on the ocean's grid and bathymetry in the `grid.nc` file. 

Use the notebooks and code from the last four classes, as well as [CoPilot](https://copilot.microsoft.com/) to select a region of the Southern Ocean and investigate some aspect of the ocean state/circulation. You are welcome to choose your own path, but you can also choose from any of the subsequent "mini" research projects.

## Comparison of temperature and salinity between West and East Antarctica

<div style="float: right; margin-right: 20px; margin-left: 30px; width: 20%; max-width: 500px;">
    <img src="images/ant_melt.png" width="100%" />
</div>

The West Antarctic Ice Sheet (WAIS) has been loosing mass in a way the East Antarctic has not (take a look at the figure from [Smith et al. 2020](https://www.science.org/doi/10.1126/science.aaz5845)). The higher vulnerability of WAIS to melt has been linked to warmer ocean temperatures at the continental shelf related to intrusions of Circumpolar Deep Water from the open ocean.

Use the temperature and salinity fields from the ACCESS-OM2 model to compare the temperature and salinity of the continental shelf at West Antarctica vs the one at East Antarctica. For this you could:
 - Do TS diagrams
 - Plot maps of temperature/salinity at different depths
 - Etc

In order to do this, you will have to be able to select the continental shelf in a region, which is slightly more nuanced than just selecting a latitude/longitude box. Take inspiration from the second class, where we looked the Antarctic Bottom Water layer selecting those densities larger than 28.27 kg/m$^3$: if the continental shelf is defined as the area shallower than 1000m, you can use the `xarray.where()` function to select the continental shelf. For example:

In [ ]:
xr.where(ht<1000, temp, np.nan)

where we are saying that where the depth `ht` is less than a 1000, we want to keep the temperature values, otherwise we want `NaN`s. 

Before doing the above, you will want to have pre-selected a `temp_WA` variable for the West Antarctic region, and a `temp_EA` for the East Antarctic region.

## Identifying Antarctic Circumpolar Current fronts from sea level